In [97]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
import math

In [2]:
# Load data
pluto = pd.read_csv('data.csv')
# Remove duplicate column names from concatenation 
pluto = pluto[pluto['Borough']!='Borough']

/Users/Laura/anaconda/lib/python2.7/site-packages/pandas/io/parsers.py:1170: DtypeWarning: Columns (1,2,3,4,5,6,7,8,10,11,12,13,18,19,21,22,23,24,29,30,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,53,55,56,57,58,59,60,61,63,64,65,66,67,68,69,70,71,72,73,74,75,76,80,81,82,84) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)


In [3]:
columns_to_remove = ['Block', 'Lot','ZoneDist1','ZoneDist2', 'ZoneDist3', 'ZoneDist4', 'Overlay1', 'Overlay2',
        'SPDist1', 'SPDist2', 'AllZoning1', 'AllZoning2','OwnerName', 'LotArea', 'BldgArea', 
        'OfficeArea', 'RetailArea', 'GarageArea', 'StrgeArea', 'FactryArea','OtherArea', 'AreaSource',
        'AssessLand', 'AssessTot', 'ExemptLand', 'ExemptTot','BuiltFAR', 'ResidFAR', 'CommFAR', 'FacilFAR',
        'ZMCode','Sanborn', 'TaxMap', 'EDesigNum', 'APPBBL', 'APPDate', 'PLUTOMapID','Address','BoroCode','Version',
        'CT2010','CB2010','SanitBoro','Tract2010', 'CD','FireComp','PolicePrct','HealthArea',
        'SanitDistrict','SanitSub']

In [4]:
for col in columns_to_remove:
    pluto = pluto.drop(col,axis=1)

In [5]:
columns_to_float =['SchoolDist', 'Council', 'ZipCode','ComArea', 'ResArea',
       'LandUse', 'Easements','NumBldgs', 'NumFloors', 'UnitsRes', 'UnitsTotal', 'LotFront',
       'LotDepth', 'BldgFront', 'BldgDepth', 'ProxCode','LotType', 'BsmtCode', 'YearBuilt',
       'YearAlter1', 'YearAlter2', 'BBL','CondoNo', 'XCoord', 'YCoord']

In [6]:
for col in columns_to_float:
    pluto[col] = pluto[col].astype(float)

In [7]:
# Remove rows with bad coordinate data
pluto = pluto[pluto['XCoord'].notnull()]
pluto = pluto[pluto['YCoord'].notnull()]
pluto = pluto[pluto['XCoord']!=0]
pluto = pluto[pluto['YCoord']!=0]

In [8]:
pluto['GrossSquareFeetPluto'] = pluto['ResArea'] + pluto['ComArea']
pluto = pluto[pluto['GrossSquareFeetPluto']!=0]

In [9]:
BinaryDict = {'N': 0, 'Y': 1}
# Split Zone Binary
pluto.replace({"SplitZone": BinaryDict},inplace=True)
# IrrLotCode Binary
pluto.replace({"IrrLotCode": BinaryDict},inplace=True)

In [10]:
# Limited Height Binary
LtdHeightDict = {'LH-1': 1, 'LH-1A': 1}
pluto.replace({"LtdHeight": LtdHeightDict},inplace=True)
pluto['LtdHeight'].fillna(value=0,inplace=True)

In [11]:
# BuiltCode Binary
BuiltCodeDict = {'E': 1}
pluto.replace({"BuiltCode": BuiltCodeDict},inplace=True)
pluto['BuiltCode'].fillna(value=0,inplace=True)

In [12]:
# Hist Dist Binary
pluto['HistDist'] = (pluto['HistDist'].notnull())*1
pluto['HistDist'].fillna(value=0,inplace=True)

In [13]:
# Landmark
pluto['Landmark'] = (pluto['Landmark'].notnull())*1
pluto['Landmark'].fillna(value=0,inplace=True)

In [14]:
# Ext New Columns
pluto['Garage'] = (pluto['Ext']==('G' or 'EG'))*1
pluto['Extension'] = (pluto['Ext']==('E' or 'EG'))*1
pluto = pluto.drop('Ext',axis=1)

In [15]:
# Count Alterations 
pluto['YearAlter1'] = (pluto['YearAlter1'] > 0)*1
pluto['YearAlter2'] = (pluto['YearAlter2'] > 0)*1
pluto['CountAlter'] = pluto['YearAlter1'] + pluto['YearAlter2']
pluto = pluto.drop('YearAlter1',axis=1)
pluto = pluto.drop('YearAlter2',axis=1)

In [90]:
# Round NumFloors and Log
pluto['NumFloors'] = pluto['NumFloors'].astype(float).round()

In [17]:
# Easements Binary
pluto['Easements'] = (pluto['Easements']>0)*1

In [18]:
# ProxCode set NaN
pluto['ProxCode'] = pluto['ProxCode'].replace(0,np.nan)

In [19]:
# BsmtCode Binary
pluto['BsmtCode'] = pluto['BsmtCode'].replace(5,np.nan)
pluto['BsmtCode'] = (pluto['BsmtCode'] > 0)*1

In [20]:
# Limit NumBldgs
pluto['NumBldgs'] = ((pluto['NumBldgs']<10)*1).replace(0,np.nan)* pluto['NumBldgs']

In [78]:
# Limit Front and Depth
pluto['LotFront'] = ((pluto['LotFront']<100)*1).replace(0,np.nan)* pluto['LotFront']
pluto['LotDepth'] = ((pluto['LotDepth']<200)*1).replace(0,np.nan)* pluto['LotDepth']
pluto['BldgFront'] = ((pluto['BldgFront']<100)*1).replace(0,np.nan)* pluto['BldgFront']
pluto['BldgDepth'] = ((pluto['BldgDepth']<200)*1).replace(0,np.nan)* pluto['BldgDepth']

In [22]:
# Fix impossible years
pluto['YearBuilt'] = ((pluto['YearBuilt']<2016)*1).replace(0,np.nan)* pluto['YearBuilt']

In [23]:
# Limit UnitRes and UnitsTotal
pluto['UnitsRes'] = ((pluto['UnitsRes']<100)*1).replace(0,np.nan)* pluto['UnitsRes']
pluto['UnitsTotal'] = ((pluto['UnitsTotal']<100)*1).replace(0,np.nan)* pluto['UnitsTotal']

In [26]:
# To make dummy:
# - Borough
# - SchoolDist
# - Council
# - BldgClass
# - LandUse
# - OwnerType
# - ProxCode
# - LotType

# Binary: 
# - LtdHeight
# - SplitZone
# - Easements
# - IrrLotCode
# - BsmtCode
# - BuiltCode
# - HistDist
# - Landmark

# Continuous:
# - NumBldgs 
# - NumFloors
# - LotFront 
# - LotDepth
# - BldgFront
# - BldgDepth
# - YearBuilt
# - UnitsRes 
# - UnitsTotal 
# - XCoord ***
# - YCoord ***

# Other:
# - BBL <- primary key
# - CondoNo
# - Zipcode